In [1]:
# Copyright © 2020-2021 Intel Corporation.
# 
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express 
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy, 
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are 
# expressly stated in the License.

# Import modules

In [2]:
import sys, os
sys.path.append(os.path.expanduser("~") + "/nxsdk-nxsdk")
import time
import numpy as np
import nxsdk.api.n2a as nx
import nxsdk_modules.slayer.src as nxSlayer
from nmnistDataset import Event

os.environ['SLURM'] = '1' 
os.environ['PARTITION'] = 'nahuku32'

# Define dataset

This is defined here just for demo. Actually __you are supposed to import the dataset module that you defined during SLAYER training__.

In [3]:
class NMNISTDataset():
    """Interface to NMNIST Test Dataset
    """
    def __init__(self, path):
        self.path = path
        self.labels = np.loadtxt(self.path+'/labels.txt').astype(int)
            
    def __getitem__(self, index):
        filename = self.path + '/' + str(index+1).zfill(5) + '.bin'
        data = Event(filename)
        label = self.labels[index]
        npEvent = np.zeros((len(data.x), 4))
        npEvent[:,0] = data.x
        npEvent[:,1] = data.y
        npEvent[:,2] = data.p
        npEvent[:,3] = data.t

        return npEvent, label

    def __len__(self):
        return len(self.labels)



# Create nxSlayer Dataset instance

In [4]:
dataset = nxSlayer.auto.s2lDataset(NMNISTDataset(path='/nfs/ncl/datasets/NMNIST_test'))

# Create network

In [5]:
net = nxSlayer.auto.Network('nmnist.net')

# Compile, setup input-output, and run the network

In [6]:
boardName = 'nmnist'
loadSavedBoard = False # True # load precompiledBoard or compile again

net.create()

if loadSavedBoard is True:
    net.compile(boardName)
    net.setupIO(dataset, numSnips=3)
else:
    net.compile()
    net.setupIO(dataset, numSnips=3)
    net.save(boardName) # this will save the compiled board
    
results = net.run()

Creating Network
|   Type   |  W  |  H  |  C  | ker | str | pad |delay|neurons/Core|cores|
|input     |   34|   34|    2|     |     |     |False|         341|    7|
|dense     |    1|    1|  512|     |     |     |False|          62|    9|
|dense     |    1|    1|   10|     |     |     |False|         277|    1|
|Total                                                            |   17|

Starting compilation ...
Completed compilation in 93.69 seconds
Using per sample spike time: 350steps (+ 100steps gap)
INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.92:34905
INFO:DRV:      Host server up..............Done 0.67s
INFO:DRV:      Encoding axons/synapses.....Done 1.78s
INFO:DRV:      Compiling Embedded snips....Done 0.97s
INFO:DRV:      Compiling MPDS Registers....Done 0.67ms
INFO:HST:  Config /etc/nx/pio.cfg
INFO:HST:  Args chip=0 cpu=0 /home/sshresth/nxsdk-nxsdk/nxsdk/driver/compilers/../../../temp/1612580555.0209997/launcher_chip0_lmt0.bin --chips=1 --remot

# Check accuracy

In [7]:
labels = dataset.labels
accuracy = nxSlayer.s2l.checkAccuracy(labels, results)
print('Final accuracy is {:.2f}%'.format(accuracy*100))

Final accuracy is 98.49%
